In [1]:
import pandas as pd, numpy as np
import os
import matplotlib.pyplot as plt
from datetime import timedelta,date
from dateutil.parser import parse
from ResearchML import *
from mlfinlab.microstructural_features import *
from mlfinlab.sample_weights import *

workPath = "/home/xetra/AlphaGoldmine/Data/"
future = "CL"
# y = 2018

contFuture = os.path.join(workPath, future.lower(), f"{future}Cont")
contFutureMLOLD = os.path.join(workPath, future.lower(), f"{future}ContML")

vbPath = os.path.join(workPath, future.lower(), f"{future}ContVB")
contFutureLevelBucketsVB = os.path.join(workPath,future.lower(), f"{future}ContLevelBuckets")
vpDir = os.path.join(workPath,future.lower(),"VP")

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100


snippedFrom = 2016


In [2]:
monthMap = {
    "F": 1,
    "G": 2,
    "H": 3,
    "J": 4,
    "K": 5,
    "M": 6,
    "N": 7,
    "Q": 8,
    "U": 9,
    "V": 10,
    "X": 11,
    "Z": 12,
}

cols = {
#     'Date' : "date",
#     ' Time' : "time",
    ' Open' : "o",
    ' High' : "h",
    ' Low' : "l",
    ' Last' : "c",
    ' Volume' : "vol",
    ' # of Trades' : "nrT",
    ' Bid Volume' : "bidVol",
    ' Ask Volume' : "askVol",
    'Date_ Time' : "date",
    "contract": "contract"
}

In [3]:
fileNames = []
for i in range(10, 21, 1):
    for m in monthMap:
        try: fileNames.append(os.path.join(workPath, future.lower(), f"{future}{m}{i}.csv"))
        except(FileNotFoundError): continue

In [4]:
def getDf(p):
    s = f"{future}{p[len(p)-7:-4]}"
    df = pd.read_csv(p, parse_dates=[["Date", " Time"]]).rename(columns={"Date_ Time":"date"})
    return df.drop_duplicates(subset=["date"], keep="last").set_index("date")[" Last"].rename(s)

with mp.Pool(processes=mp.cpu_count()-1) as p:
    res = p.map(getDf, fileNames)
    df = pd.concat(res,1)

In [5]:
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

In [6]:
df=df.sort_index().reset_index()
dt = df["date"]
df = df.drop("date",1)

In [7]:
# X,Y = np.meshgrid(np.arange(len(df.columns)),df.index)
# Z = df
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(X, Y, Z)

In [8]:
df.columns

Index(['CLF10', 'CLG10', 'CLH10', 'CLJ10', 'CLK10', 'CLM10', 'CLN10', 'CLQ10',
       'CLU10', 'CLV10',
       ...
       'CLH20', 'CLJ20', 'CLK20', 'CLM20', 'CLN20', 'CLQ20', 'CLU20', 'CLV20',
       'CLX20', 'CLZ20'],
      dtype='object', length=132)

In [9]:
df["date"] = dt

In [10]:
df=df.set_index("date")

In [11]:
for c in list(df):
    df.loc[df[c].first_valid_index():df[c].last_valid_index(),c].fillna(method="ffill", inplace=True)

In [12]:
cAsDate={k:v for k,v in [f for f in zip([f for f in df.columns],[date(2000+int(c[3:]),monthMap[c[2]],20) for c in df.columns])]}

In [20]:
for i,row in df.iterrows():
    r=row.dropna()
    tau=abs(cAsDate["CLF10"]-r.name.date())/timedelta(days=1)
    print(tau)
    break

625.0


In [ ]:
n = "CLG11"
m=n[2]
y=int(n[3:])
d = 20

In [ ]:
y

In [ ]:
date(2000+y,monthMap[m],d)

In [ ]:
df.iloc[200000].dropna()